In [1]:
using Revise
using DualOptimalFiltering
using Distributions, Random, BenchmarkTools, Profile, ProfileView, JLD



┌ Info: Recompiling stale cache file /home/guillaume/.julia/compiled/v1.0/DualOptimalFiltering/EOzNm.ji for DualOptimalFiltering [4abc1bc0-c194-11e8-24a1-c168f2b1c514]
└ @ Base loading.jl:1190
Expr(:const, Symbol("##λm_mem_memoized_cache") = Expr(:call, :IdDict))
  ** incremental compilation may be broken for this module **

Expr(:const, Symbol("##logλm_mem_memoized_cache") = Expr(:call, :IdDict))
  ** incremental compilation may be broken for this module **

Expr(:const, Symbol("##denominator_Cmmi_mem_memoized_cache") = Expr(:call, :IdDict))
  ** incremental compilation may be broken for this module **

Expr(:const, Symbol("##logCmmi_mem_memoized_cache") = Expr(:call, :IdDict))
  ** incremental compilation may be broken for this module **

Expr(:const, Symbol("##logCmmi_mem_overflow_safe_memoized_cache") = Expr(:call, :IdDict))
  ** incremental compilation may be broken for this module **




Welcome to Nemo version 0.11.1

Nemo comes with absolutely no warranty whatsoever


Welcome to Nemo version 0.11.1

Nemo comes with absolutely no warranty whatsoever



Expr(:const, Symbol("##log_binomial_safe_but_slow_mem_memoized_cache") = Expr(:call, :IdDict))
  ** incremental compilation may be broken for this module **

┌ Warning: RCall.jl: Loading tidyverse: ggplot2
│ Loading tidyverse: tibble
│ Loading tidyverse: tidyr
│ Loading tidyverse: readr
│ Loading tidyverse: purrr
│ Loading tidyverse: dplyr
│ Conflicts with tidy packages ---------------------------------------------------
│ filter(): dplyr, stats
│ lag():    dplyr, stats
└ @ RCall ~/.julia/packages/RCall/RPlFw/src/io.jl:110



Welcome to Nemo version 0.11.1

Nemo comes with absolutely no warranty whatsoever



In [2]:
function simulate_WF3_data()
    K = 3
    α = ones(K)
    sα = sum(α)
    Pop_size_WF3 = 15
    Ntimes_WF3 = 5
    time_step_WF3 = 0.1
    Random.seed!(4)
    wfchain_WF3 = rand(Dirichlet(K,0.3)) |> z-> DualOptimalFiltering.wright_fisher_PD1(z, 1.5, 50, Ntimes_WF3)[:,2:end]
    wfobs_WF3 = [rand(Multinomial(Pop_size_WF3, wfchain_WF3[:,k])) for k in 1:size(wfchain_WF3,2)] |> l -> hcat(l...)
    time_grid_WF3 = [k*time_step_WF3 for k in 0:(Ntimes_WF3-1)]
    data_WF3 = Dict(zip(time_grid_WF3 , [wfobs_WF3[:,t:t]' |> collect for t in 1:size(wfobs_WF3,2)]))
    return data_WF3, wfchain_WF3, α
end
data, wfchain_WF3, α = simulate_WF3_data()
times = data |> keys |> collect |> sort

5-element Array{Float64,1}:
 0.0                
 0.1                
 0.2                
 0.30000000000000004
 0.4                

In [3]:
log_ν_dict, log_Cmmi_dict, log_binomial_coeff_dict = DualOptimalFiltering.precompute_log_terms_arb(data, sum(α); digits_after_comma_for_time_precision = 4, override = true)

@btime DualOptimalFiltering.filter_WF_precomputed(α, data, log_ν_dict, log_Cmmi_dict, log_binomial_coeff_dict; silence = true)

Precomputing 3 times
2.775000e+03 terms
  3.859 s (30864538 allocations: 2.05 GiB)


(Dict{Any,Any}(0.0=>Array{Int64,1}[[2, 10, 3]],0.4=>Array{Int64,1}[[5, 9, 6], [2, 9, 9], [5, 30, 12], [4, 26, 17], [5, 14, 9], [8, 35, 12], [3, 42, 6], [6, 43, 19], [4, 17, 16], [5, 9, 10]  …  [1, 8, 18], [8, 23, 20], [3, 36, 14], [7, 30, 9], [1, 41, 19], [7, 40, 12], [5, 15, 7], [1, 14, 13], [4, 33, 7], [7, 11, 19]],0.2=>Array{Int64,1}[[2, 9, 9], [4, 15, 13], [3, 18, 12], [3, 18, 8], [3, 23, 11], [2, 11, 6], [1, 14, 8], [2, 12, 9], [4, 25, 8], [2, 28, 6]  …  [4, 16, 10], [4, 19, 6], [1, 21, 7], [3, 14, 12], [3, 11, 13], [2, 26, 7], [1, 12, 11], [1, 14, 13], [3, 21, 6], [1, 22, 10]],0.3=>Array{Int64,1}[[4, 11, 5], [7, 11, 2], [7, 32, 8], [6, 34, 12], [6, 28, 13], [4, 25, 15], [7, 16, 5], [6, 17, 9], [5, 20, 8], [6, 19, 12]  …  [7, 17, 3], [6, 21, 14], [3, 16, 9], [5, 12, 14], [5, 23, 2], [4, 31, 12], [7, 18, 5], [3, 31, 4], [6, 35, 3], [3, 24, 6]],0.1=>Array{Int64,1}[[2, 18, 5], [2, 10, 7], [2, 11, 7], [2, 13, 5], [3, 20, 4], [2, 16, 5], [2, 14, 4], [3, 14, 6], [2, 17, 7], [3, 13, 5]  

In [4]:
@btime DualOptimalFiltering.filter_WF_adaptive_precomputation(α, data, (x,y) -> (x,y); silence = true)

  4.270 s (31450495 allocations: 2.09 GiB)


(Dict{Any,Any}(0.0=>Array{Int64,1}[[2, 10, 3]],0.4=>Array{Int64,1}[[5, 9, 6], [2, 9, 9], [5, 30, 12], [4, 26, 17], [5, 14, 9], [8, 35, 12], [3, 42, 6], [6, 43, 19], [4, 17, 16], [5, 9, 10]  …  [1, 8, 18], [8, 23, 20], [3, 36, 14], [7, 30, 9], [1, 41, 19], [7, 40, 12], [5, 15, 7], [1, 14, 13], [4, 33, 7], [7, 11, 19]],0.2=>Array{Int64,1}[[2, 9, 9], [4, 15, 13], [3, 18, 12], [3, 18, 8], [3, 23, 11], [2, 11, 6], [1, 14, 8], [2, 12, 9], [4, 25, 8], [2, 28, 6]  …  [4, 16, 10], [4, 19, 6], [1, 21, 7], [3, 14, 12], [3, 11, 13], [2, 26, 7], [1, 12, 11], [1, 14, 13], [3, 21, 6], [1, 22, 10]],0.3=>Array{Int64,1}[[4, 11, 5], [7, 11, 2], [7, 32, 8], [6, 34, 12], [6, 28, 13], [4, 25, 15], [7, 16, 5], [6, 17, 9], [5, 20, 8], [6, 19, 12]  …  [7, 17, 3], [6, 21, 14], [3, 16, 9], [5, 12, 14], [5, 23, 2], [4, 31, 12], [7, 18, 5], [3, 31, 4], [6, 35, 3], [3, 24, 6]],0.1=>Array{Int64,1}[[2, 18, 5], [2, 10, 7], [2, 11, 7], [2, 13, 5], [3, 20, 4], [2, 16, 5], [2, 14, 4], [3, 14, 6], [2, 17, 7], [3, 13, 5]  

In [5]:
@btime DualOptimalFiltering.WF_loglikelihood_adaptive_precomputation($α, $data; silence = true)

  22.307 s (133171445 allocations: 71.88 GiB)


Dict{Float64,Float64} with 5 entries:
  0.0 => -4.91265
  0.4 => -21.0496
  0.2 => -12.5022
  0.3 => -16.8908
  0.1 => -8.69155

In [6]:
@btime DualOptimalFiltering.WF_loglikelihood_adaptive_precomputation_pruning($α, $data, (x,y) -> (x,y); silence = true)

  10.923 s (105926818 allocations: 7.68 GiB)


Dict{Float64,Float64} with 5 entries:
  0.0 => -4.91265
  0.4 => -21.0496
  0.2 => -12.5022
  0.3 => -16.8908
  0.1 => -8.69155